# Calibration Notebook
> Guan, Shenheng, Michael F. Moran, et Bin Ma. « Prediction of LC-MS/MS Properties of Peptides from Sequence by Deep Learning ». Molecular & Cellular Proteomics : MCP 18, no 10 (octobre 2019): 2099‑2107. https://doi.org/10.1074/mcp.TIR119.001412.

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import random
from pyteomics import mass
from pyteomics import electrochem

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error

import joblib


In [27]:
__path_model__ = "D:\LCMSMS_Pred_Supplemental_Material_section_S6\LCMSMS_Pred_Supplemental_Material_section_S6\ChargeState\zfit_bidirLSTM2_masking_model.h5"
__DATA_1_PATH__ = "D:\dev\Stats\data\dic.pkl"
__DATA_2_PATH__ = "D:\dev\RT_STUDY\Quantified_peptide_ions.tsv"

### Our data

In [101]:
dic = joblib.load(__DATA_1_PATH__)

def generate_data(dic):
    charge = []
    seq = []

    for i in dic.keys():
        for j in dic[i].keys():
            if j != 'total':
                charge.append(str(j))
                seq.append(dic[i][j]['sequence'])
    return np.array(seq),np.array(charge)

seq,charge = generate_data(dic)
print(len(seq),len(charge))

names = ''' peptide_id	sequence	modifications	master_elution_time	master_quant_peptide_ion_moz	master_quant_peptide_ion_charge	master_quant_peptide_ion_elution_time	master_quant_peptide_ion_feature_id	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72878	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72882	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72898	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72904	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72907	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72911	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73324	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73334	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73344	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73495	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73500	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73578	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73592	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73599	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73622	psm_count_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73631	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72878	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72882	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72898	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72904	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72907	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72911	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73324	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73334	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73344	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73495	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73500	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73578	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73592	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73599	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73622	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73631	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72878	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72882	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72898	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72904	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72907	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72911	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73324	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73334	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73344	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73495	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73500	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73578	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73592	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73599	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73622	corrected_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73631	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72878	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72882	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72898	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72904	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72907	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72911	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73324	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73334	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73344	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73495	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73500	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73578	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73592	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73599	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73622	raw_abundance_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73631'''

elution_time_new_file = '''elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72878	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72882	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72898	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72904	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72907	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72911	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73324	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73334	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73344	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73495	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73500	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73578	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73592	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73599	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73622	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73631'''

# elution_time_new_file = elution_time_new_file.replace(' ','_')
# elution_time_new_file = elution_time_new_file.replace('-','_')
elution_time_new_file = elution_time_new_file.split('\t')


labels = '''elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72878	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72882	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72898	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72904	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72907	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_72911	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73324	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73334	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73344	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73495	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73500	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73578	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73592	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73599	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73622	elution_time_MC HeLa Thermo MS60K-HCD15KG120 1ug ON 45C_73631	'''
labels = labels.replace('\t','_labels\t')
# labels = labels.replace('-','_')
# labels = labels.replace(' ','_')
labels = labels.split('\t')

df = pd.read_table(__DATA_2_PATH__, sep='\t', names=names.split('\t'))
df = df[1:]
df.head()

ions = np.array(df['master_quant_peptide_ion_charge'].values)
ions = np.concatenate((ions,charge))

seq1 = np.array(df['sequence'].values)
seq = np.concatenate((seq1,seq))

data = pd.DataFrame(list(zip(seq, ions)),
               columns =['seq', 'charge'])
print(data.shape)
data.head()

56571 56571
(121277, 2)


C:\Users\pelegrin\AppData\Local\Temp\ipykernel_7428\4150674683.py:32: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table(__DATA_2_PATH__, sep='\t', names=names.split('\t'))


,seq,charge
0,GPQVALK,2
1,GPALDIK,2
2,FSVSGLK,2
3,APSLDIK,2
4,GPEVDIK,2


### One Hot modified functions 

In [102]:
##Prediction of LCMSMS properties of peptides from sequence by deep learning
##Shenheng Guan1, Michael F. Moran, and Bin Ma
##2019-02-21

## MODIFIED BY JAVIER PELEGRIN GARCIA
##2023-07-28

import numpy as np



class InvalidPeptideLength(Exception):
    "Error peptide is too long."
    pass

class InvalidCharge(Exception):
    "Error charge is invalid."
    pass


psi_to_single_ptm = {'(Acetyl)-': 'B',
                     '(Carbamyl)': 'O',
                     '(Carbamidomethyl)': '',
                     'M(Oxidation)': 'J',
                     '(Gln->pyro-Glu)Q': 'X',
                     'N(Deamidated)': 'D',
                     'Q(Deamidated)': 'E'}

def reshapeOneHot(X):
    X = np.dstack(X)
    X = np.swapaxes(X, 1, 2)
    X = np.swapaxes(X, 0, 1)
    return X

def get_single_ptm_code(psi_sequence):
    sequence = psi_sequence
    for ptm in psi_to_single_ptm:
        sequence = sequence.replace(ptm, psi_to_single_ptm[ptm])
    return sequence

def one_hot_encode_peptide(psi_sequence, MAX_LENGTH = 41):
    peptide = get_single_ptm_code(psi_sequence)
    if len(peptide) > MAX_LENGTH:
        # print('Peptide length is larger than maximal length of ', str(MAX_LENGTH))
        raise InvalidPeptideLength
    else:
        AA_vocabulary = 'KRPTNAQVSGILCMJHFYWEDBXOU'#B: acetyl; O: Carbamyl; J: oxidized Met; X:pyro_glu
        no_not_used_aas = 2#U: not used

        one_hot_peptide = np.zeros((len(peptide), len(AA_vocabulary) - no_not_used_aas))

        # print(one_hot_peptide.shape,(len(peptide), len(AA_vocabulary) - no_not_used_aas))

        for j in range(0, len(peptide)):
            try:
                aa = peptide[j]
                one_hot_peptide[j, AA_vocabulary.index(aa)] = 1
            except:
                pass
        
        no_front_paddings = int((MAX_LENGTH - len(peptide))/2)
        peptide_front_paddings = np.zeros((no_front_paddings, one_hot_peptide.shape[1]))

        no_back_paddings = MAX_LENGTH - len(peptide) - no_front_paddings
        peptide_back_paddings = np.zeros((no_back_paddings, one_hot_peptide.shape[1]))

        full_one_hot_peptide = np.vstack((peptide_front_paddings, one_hot_peptide, peptide_back_paddings))

        return peptide, full_one_hot_peptide
    
def one_hot_encode_peptide_ion(psi_sequence, charge, MAX_LENGTH = 41, MAX_CHARGE = 6):

    if len(psi_sequence) >= MAX_CHARGE:
        raise InvalidPeptideLength

    peptide, full_one_hot_peptide = one_hot_encode_peptide(psi_sequence)
    
    one_hot_charge = np.zeros((len(peptide), MAX_CHARGE))
    one_hot_charge[:, charge - 1] = 1
    
    no_front_paddings = int((MAX_LENGTH - len(peptide))/2)
    charge_front_paddings = np.zeros((no_front_paddings, one_hot_charge.shape[1]))

    no_back_paddings = MAX_LENGTH - len(peptide) - no_front_paddings
    charge_back_paddings = np.zeros((no_back_paddings, one_hot_charge.shape[1]))

    full_one_hot_charge = np.vstack((charge_front_paddings, one_hot_charge, charge_back_paddings))

    full_one_hot_peptide_ion = np.hstack((full_one_hot_peptide, full_one_hot_charge))

    return full_one_hot_peptide_ion


def on_hot_encode_charge(charge,MAX_CHARGE = 6):
    if charge > MAX_CHARGE:
        raise InvalidCharge
    one_hot_charge = np.zeros((1, MAX_CHARGE))
    one_hot_charge[:, charge - 1] = 1
    return one_hot_charge

In [47]:
class differentlenght(Exception):
    "List have differents length"
    pass

def normalize_data(list_seq, list_charge):
    label, one_hot = [], []
    if len(list_seq) != len(list_charge):
        print(f"list of seq: {len(list_seq)} and list of charge: {len(list_charge)} have differents length.")
        raise differentlenght
    for seq,charge in zip(list_seq,list_charge):
        try:
            one_hot.append(one_hot_encode_peptide(seq)[1])
            label.append(on_hot_encode_charge(charge))
        except InvalidPeptideLength:
            print(f"peptide {seq} with length = {len(seq)} is too long, Max length is 41")
            pass
        except InvalidCharge:
            print(f"peptide {seq} with charge = {charge} is invalid")
            one_hot.pop()
            pass
    return np.array(label),np.array(one_hot)

#### Normalize data

In [ ]:
X,y = normalize_data(data['seq'].values, data['charge'].values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('Total corpus size')
print('\t• train :', len(X_train), 'exemples')
print('\t• test :', len(X_test), 'exemples')

# Affichage de la taille des images et des labels dans le corpus 
print('\nTraining data size')
print('\t• X_train (masse,residues,charge):', X_train.shape)
print('\t• y_train (labels) :', y_train.shape)

print('\nTest data size')
print('\t• X_test (masse,residues,charge) :', X_test.shape)
print('\t• y_test (labels) :', y_test.shape)

#### Load the model from keras

In [ ]:
from keras.models import load_model

model = load_model(__path_model__)

history = model.fit(X_train, 
		      y_train, 
		      epochs=200,  # More epochs because more data
		      batch_size=1024,
		      validation_split=0.2)

cost = model.evaluate(X_test, y_test, batch_size=1024)
print('test cost:', cost)
from keras.models import load_model
model.save("Model_one_hot_calibration_my_data.h5")

Start predictions with test data

In [ ]:
predict_charge = model.predict(X_test)

In [ ]:
f = lambda x : np.argmax(x)+1
predict_y = np.array(list(map(f,predict_charge)))

charge = np.array(list(map(f,y_test)))

y_pred = [[],[],[],[]]
for pred,real in zip(predict_y,charge):
    y_pred[real-2].append(pred)

# print(y_pred)
print(f'RMSE: {mean_squared_error(charge, predict_y, squared=False)}')
plt.violinplot(y_pred)
plt.xticks([1,2,3,4],['2+','3+','4+','5+'])
plt.xlabel('Real charge')
plt.ylabel('Prediced charge')
plt.title('Model of Prediction of LC-MS/MS Properties of Peptides from Sequence by Deep Learning')
plt.show()